In [8]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Indian Dataset

In [9]:
# For Indian Sign Language Dataset (No predefined train/test split)
# Assumes structure: dataset/Indian/A/, dataset/Indian/B/, ..., dataset/Indian/1/, ..., dataset/Indian/9/

data_dir = r"D:\NULL CLASS\Task 3\Indian"
img_size = 64  # you can choose any suitable size
images = []
labels = []

for label in os.listdir(data_dir):
    label_path = os.path.join(data_dir, label)
    if os.path.isdir(label_path):
        for img_name in os.listdir(label_path):
            img_path = os.path.join(label_path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (img_size, img_size))
                images.append(img)
                labels.append(label)

images = np.array(images)
labels = np.array(labels)

In [10]:
# Labels are already inferred from folder names in the image loading step
# But you can convert them to numeric labels using this mapping:

from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
numeric_labels = label_encoder.fit_transform(labels)

In [11]:
# Resize is already done during image loading; now normalize the images
images = images.astype('float32') / 255.0

In [16]:
X_train, X_val, y_train, y_val = train_test_split(images, numeric_labels, test_size=0.2, random_state=42, stratify=numeric_labels)

In [17]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)

In [18]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y_train.shape[1], activation='softmax')
])

In [19]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

Epoch 1/20
350/350 [==============================] - 49s 136ms/step - loss: 0.5341 - accuracy: 0.8464 - val_loss: 0.0041 - val_accuracy: 0.9993
Epoch 2/20
350/350 [==============================] - 49s 140ms/step - loss: 0.0532 - accuracy: 0.9816 - val_loss: 0.0044 - val_accuracy: 0.9986
Epoch 3/20
350/350 [==============================] - 47s 134ms/step - loss: 0.0330 - accuracy: 0.9901 - val_loss: 4.1176e-04 - val_accuracy: 1.0000
Epoch 4/20
350/350 [==============================] - 48s 137ms/step - loss: 0.0228 - accuracy: 0.9932 - val_loss: 0.0010 - val_accuracy: 0.9993
Epoch 5/20
350/350 [==============================] - 48s 138ms/step - loss: 0.0193 - accuracy: 0.9937 - val_loss: 5.1570e-05 - val_accuracy: 1.0000
Epoch 6/20
350/350 [==============================] - 49s 140ms/step - loss: 0.0191 - accuracy: 0.9932 - val_loss: 2.8907e-05 - val_accuracy: 1.0000
Epoch 7/20
350/350 [==============================] - 49s 140ms/step - loss: 0.0137 - accuracy: 0.9956 - val_loss: 8.6

In [10]:
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

88/88 [==============================] - 2s 25ms/step - loss: 2.2220e-04 - accuracy: 1.0000
Validation Loss: 0.00022220012033358216
Validation Accuracy: 1.0


In [11]:
# Final evaluation on validation set (same as previous step)
final_loss, final_accuracy = model.evaluate(X_val, y_val)
print(f"Final Loss: {final_loss}")
print(f"Final Accuracy: {final_accuracy}")

88/88 [==============================] - 2s 26ms/step - loss: 2.2220e-04 - accuracy: 1.0000
Final Loss: 0.00022220012033358216
Final Accuracy: 1.0


In [20]:
model.save("indian_sign_language_model.h5")

C:\Users\VISHWA\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [21]:
from datetime import datetime

def is_within_active_hours():
    current_time = datetime.now().time()
    start_time = datetime.strptime("18:00", "%H:%M").time()
    end_time = datetime.strptime("22:00", "%H:%M").time()
    return start_time <= current_time <= end_time

In [22]:
from tensorflow.keras.models import load_model

model = load_model("indian_sign_language_model.h5")

GUI

In [24]:
import tkinter as tk
from tkinter import filedialog, Label, Button
import cv2
from PIL import Image, ImageTk

def upload_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        img = cv2.imread(file_path)
        img = cv2.resize(img, (64, 64))
        img = img.astype('float32') / 255.0
        img = img.reshape(1, 64, 64, 3)
        if is_within_active_hours():
            prediction = model.predict(img)
            predicted_label = label_encoder.inverse_transform([prediction.argmax()])[0]
            result_label.config(text=f"Prediction: {predicted_label}")
        else:
            result_label.config(text="Access only allowed between 6 PM and 10 PM")

def start_video():
    global cap
    cap = cv2.VideoCapture(0)
    show_frame()

def stop_video():
    global cap
    cap.release()
    video_label.config(image='')

def show_frame():
    if cap.isOpened():
        ret, frame = cap.read()
        if ret:
            img = cv2.resize(frame, (64, 64))
            img_normalized = img.astype('float32') / 255.0
            img_normalized = img_normalized.reshape(1, 64, 64, 3)
            if is_within_active_hours():
                prediction = model.predict(img_normalized)
                predicted_label = label_encoder.inverse_transform([prediction.argmax()])[0]
                result_label.config(text=f"Prediction: {predicted_label}")
            else:
                result_label.config(text="Time restricted: 6 PM - 10 PM")

            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            video_label.imgtk = imgtk
            video_label.configure(image=imgtk)
        video_label.after(10, show_frame)

root = tk.Tk()
root.title("Sign Language Detector")

upload_btn = Button(root, text="Upload Image", command=upload_image)
upload_btn.pack()

start_btn = Button(root, text="Start Video", command=start_video)
start_btn.pack()

stop_btn = Button(root, text="Stop Video", command=stop_video)
stop_btn.pack()

video_label = Label(root)
video_label.pack()

result_label = Label(root, text="Prediction will appear here")
result_label.pack()

root.mainloop()